In [ ]:
!pip install flask-restful google-cloud-storage
import nltk
nltk.download('stopwords')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 4.7 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# authenticate below for Google Storage access as needed
from google.colab import auth
auth.authenticate_user()

# Run the app

In [ ]:
# you need to upload your implementation of search_app.py
import search_frontend as se

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Initializing Backend - VERSION: V8_LOG_ANCHORS
Loading indices...
Backend ready!


In [ ]:
# find Colab's public URL
from google.colab.output import eval_js
server_url = eval_js("google.colab.kernel.proxyPort(5000)")
print(f"""Test your search engine by navigating to
{server_url}search?query=hello+world
This URL is only accessible from the same browser session. In other words, this
will not be accessible from a different machine, browser, or incognito session.
""")

# Uncomment the following line of code to run the frontend in the main
# process and wait for HTTP requests (colab will hang). The debug parameter
# lets you see incoming requests and get debug print outs if exceptions occur.
# se.run(debug=False, use_reloader=False)

# Alternatively, the next few lines run the frontend in a background process.
# Just don't forget to terminate the process when you update your search engine
# or want to reload it.
import multiprocessing, time
proc = multiprocessing.Process(target=se.run,
                               kwargs={"debug": True, "use_reloader": False,
                                       "host": "0.0.0.0", "port": 5000})
proc.start()

time.sleep(1) # give Flask time to boot

from google.colab.output import eval_js
server_url = eval_js("google.colab.kernel.proxyPort(5000)")

print(f"Open this URL:\n{server_url}/search?query=hello+world")
# Use proc.terminate() to stop the process

Test your search engine by navigating to
https://5000-m-s-18sshwzl2yi74-a.us-east4-2.prod.colab.devsearch?query=hello+world
This URL is only accessible from the same browser session. In other words, this
will not be accessible from a different machine, browser, or incognito session.

 * Serving Flask app 'search_frontend'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


Open this URL:
https://5000-m-s-18sshwzl2yi74-a.us-east4-2.prod.colab.dev/search?query=hello+world


In [ ]:
import requests

res = requests.get('http://127.0.0.1:5000/search?query=hello')

if res.status_code == 200:
    print(res.json())
else:
    print(f"Server Error (Status {res.status_code}):")
    # This prints the actual error message hidden in the HTML
    print(res.text)

[['99032', 'Hello, Dolly! (musical)'], ['54295', 'Hello Kitty'], ['1122016', 'Hello! (magazine)'], ['48324759', 'Hello (Adele song)'], ['1241470', 'Hello! Project'], ['13834', '"Hello, World!" program'], ['7024876', 'Hello, Dolly! (film)'], ['6710844', 'Hello'], ['1206479', 'Hello, Goodbye'], ['5829200', 'Hello (Lionel Richie song)'], ['1895881', 'Hello, Dolly! (song)'], ['35525195', 'Hello Venus'], ['710794', 'Hello Nasty'], ['3162118', 'Hello, Larry'], ['28063614', 'List of Hello Kitty television series'], ['899002', 'Hello Sailor (band)'], ['29972030', 'Hello (Martin Solveig song)'], ['1379058', 'Hello Muddah, Hello Fadduh (A Letter from Camp)'], ['43449879', 'Hello, My Name Is Doris'], ['479470', 'Hello, I Must Be Going! (album)'], ['54503643', 'Hello Sunshine (company)'], ['47270773', 'Hello Counselor'], ['19924718', "Hello Kitty's Furry Tale Theater"], ['28935567', 'Hello Games'], ['6225145', 'Hello, I Love You'], ['39640246', 'Hello Ladies'], ['3047381', 'Hello Mary Lou'], ['464

In [ ]:
def precision_at_k(true_list, predicted_list, k):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    if len(predicted_list) == 0:
        return 0.0
    return round(len([1 for doc_id in predicted_list if doc_id in true_set]) / len(predicted_list), 3)

def recall_at_k(true_list, predicted_list, k):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    if len(true_set) < 1:
        return 1.0
    return round(len([1 for doc_id in predicted_list if doc_id in true_set]) / len(true_set), 3)

def f1_at_k(true_list, predicted_list, k):
    p = precision_at_k(true_list, predicted_list, k)
    r = recall_at_k(true_list, predicted_list, k)
    if p == 0.0 or r == 0.0:
        return 0.0
    return round(2.0 / (1.0/p + 1.0/r), 3)

def results_quality(true_list, predicted_list):
    p5 = precision_at_k(true_list, predicted_list, 5)
    f1_30 = f1_at_k(true_list, predicted_list, 30)
    if p5 == 0.0 or f1_30 == 0.0:
        return 0.0
    return round(2.0 / (1.0/p5 + 1.0/f1_30), 3)

In [ ]:
import json
import requests
from time import time

# 1. Load ALL queries
with open('queries_train.json', 'rt') as f:
    all_queries = json.load(f)

# 2. Split the data (List conversion preserves order)
all_items = list(all_queries.items())

# First 20 queries for "Training" (Tuning weights)
train_queries = dict(all_items[:20])

# Last 10 queries for "Testing" (Held-out validation)
test_queries = dict(all_items[20:])

print(f"Loaded {len(all_queries)} queries.")
print(f"Training Set: {len(train_queries)} queries")
print(f"Test Set:     {len(test_queries)} queries")

def evaluate_engine(query_set):
    """Runs the provided set of queries and calculates Average Precision score."""
    total_score = 0
    count = 0

    # We iterate over the specific query_set passed to the function
    for q, true_wids in query_set.items():
        try:
            # Send request to your local server
            res = requests.get('http://127.0.0.1:5000/search', {'query': q}, timeout=35)

            if res.status_code == 200:
                results = res.json()
                if len(results) > 0:
                    pred_wids, _ = zip(*results)
                else:
                    pred_wids = []

                # Use the quality metric defined earlier in the notebook
                # Note: Ensure 'results_quality' or 'average_precision' is defined in your Colab
                score = results_quality(true_wids, pred_wids)

                total_score += score
                count += 1
            else:
                print(f"Server returned error for '{q}': {res.status_code}")

        except Exception as e:
            print(f"Error on query '{q}': {e}")

    return total_score / count if count > 0 else 0

# --- EXPERIMENT LOOP ---

print("\n--------------------------------------------------")
print("Evaluating on training set")
train_score = evaluate_engine(train_queries)
print(f" >> Training Score: {train_score:.4f}")
print("--------------------------------------------------")

print("\n--------------------------------------------------")
print("FINAL CHECK: Evaluating on TEST set...")
test_score = evaluate_engine(test_queries)
print(f" >> Test Score: {test_score:.4f}")
print("--------------------------------------------------")

Loaded 30 queries.
Training Set: 20 queries
Test Set:     10 queries

--------------------------------------------------
Evaluating on training set
 >> Training Score: 0.3624
--------------------------------------------------

--------------------------------------------------
FINAL CHECK: Evaluating on TEST set...
 >> Test Score: 0.2928
--------------------------------------------------


# Testing your app

Once your app is running you can query it. You can simply do that by clicking on the URL printed above (the one looking like https://XXXXX-5000-colab.googleusercontent.com/search?query=hello+world or by issuing an HTTP request through code (from colab).

The code below shows how to issue a query from python. This is also how our testing code will issue queries to your search engine, so make sure to test your search engine this way after you deploy it to GCP and before submission. Command line instructions for deploying your search engine to GCP are available at `run_frontend_in_gcp.sh`. Note that we will not only issue training queries to your search engine, but also test queries, i.e. queries that you've never seen before.

In [ ]:
import json

with open('queries_train.json', 'rt') as f:
  queries = json.load(f)

In [ ]:
def average_precision(true_list, predicted_list, k=40):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    precisions = []
    for i,doc_id in enumerate(predicted_list):
        if doc_id in true_set:
            prec = (len(precisions)+1) / (i+1)
            precisions.append(prec)
    if len(precisions) == 0:
        return 0.0
    return round(sum(precisions)/len(precisions),3)

In [ ]:
def precision_at_k(true_list, predicted_list, k):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    if len(predicted_list) == 0:
        return 0.0
    return round(len([1 for doc_id in predicted_list if doc_id in true_set]) / len(predicted_list), 3)
def recall_at_k(true_list, predicted_list, k):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    if len(true_set) < 1:
        return 1.0
    return round(len([1 for doc_id in predicted_list if doc_id in true_set]) / len(true_set), 3)
def f1_at_k(true_list, predicted_list, k):
    p = precision_at_k(true_list, predicted_list, k)
    r = recall_at_k(true_list, predicted_list, k)
    if p == 0.0 or r == 0.0:
        return 0.0
    return round(2.0 / (1.0/p + 1.0/r), 3)
def results_quality(true_list, predicted_list):
    p5 = precision_at_k(true_list, predicted_list, 5)
    f1_30 = f1_at_k(true_list, predicted_list, 30)
    if p5 == 0.0 or f1_30 == 0.0:
        return 0.0
    return round(2.0 / (1.0/p5 + 1.0/f1_30), 3)

assert precision_at_k(range(10), [1,2,3] , 2) == 1.0
assert recall_at_k(   range(10), [10,5,3], 2) == 0.1
assert precision_at_k(range(10), []      , 2) == 0.0
assert precision_at_k([],        [1,2,3],  5) == 0.0
assert recall_at_k(   [],        [10,5,3], 2) == 1.0
assert recall_at_k(   range(10), [],       2) == 0.0
assert f1_at_k(       [],        [1,2,3],  5) == 0.0
assert f1_at_k(       range(10), [],       2) == 0.0
assert f1_at_k(       range(10), [0,1,2],  2) == 0.333
assert f1_at_k(       range(50), range(5), 30) == 0.182
assert f1_at_k(       range(50), range(10), 30) == 0.333
assert f1_at_k(       range(50), range(30), 30) == 0.75
assert results_quality(range(50), range(5))  == 0.308
assert results_quality(range(50), range(10)) == 0.5
assert results_quality(range(50), range(30)) == 0.857
assert results_quality(range(50), [-1]*5 + list(range(5,30))) == 0.0


In [ ]:
import requests
from time import time
# In GCP the public URL for your engine should look like this:
# url = 'http://35.232.59.3:8080'
# In colab, we are going to send HTTP requests to localhost (127.0.0.1)
# and direct them to port where the server is listening (5000).
url = 'http://127.0.0.1:5000'

qs_res = []
for q, true_wids in queries.items():
  duration, ap = None, None
  t_start = time()
  try:
    res = requests.get(url + '/search', {'query': q}, timeout=35)
    duration = time() - t_start
    if res.status_code == 200:
      pred_wids, _ = zip(*res.json())
      rq = results_quality(true_wids, pred_wids)
  except:
    pass

  qs_res.append((q, duration, rq))